In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.microsoft import EdgeChromiumDriverManager
import time

# Setup Selenium
service = Service(EdgeChromiumDriverManager().install())
options = Options()
options.add_argument("--start-maximized")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

driver = webdriver.Edge(service=service, options=options)

# Daftar query pencarian
queries = ["2024", "2023", "2022", "2021", "2020", "2019", "2018", "2017", "2016", "2015"]
max_pages = 100
titles_list = []

# Looping melalui setiap query
for q in queries:
    for page in range(max_pages):
        start = page * 10  # Google Scholar menampilkan 10 artikel per halaman
        search_url = f"https://scholar.google.com/scholar?q={q}&start={start}"
        
        driver.get(search_url)
        time.sleep(5)  # Tambahkan delay lebih lama untuk menghindari deteksi bot

        # Debug: Periksa apakah halaman termuat dengan benar
        page_source = driver.page_source
        if "did not match any articles" in page_source:
            print(f"Hasil pencarian kosong di halaman {page + 1}, hentikan scraping.")
            break

        try:
            # Tunggu elemen artikel muncul
            wait = WebDriverWait(driver, 10)
            wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "h3.gs_rt")))

            # Ambil semua judul artikel
            titles = driver.find_elements(By.CSS_SELECTOR, "h3.gs_rt")
            if not titles:
                print(f"Tidak ada judul di halaman {page + 1}, mungkin Google memblokir akses.")
                break

            for title in titles:
                if title.text.strip():
                    titles_list.append(title.text.strip())

            print(f"Halaman {page + 1} selesai, total artikel terkumpul: {len(titles_list)}")

        except Exception as e:
            print(f"Error di halaman {page + 1}: {e}")
            break



# Tutup browser
driver.quit()

Halaman 1 selesai, total artikel terkumpul: 10
Halaman 2 selesai, total artikel terkumpul: 20
Halaman 3 selesai, total artikel terkumpul: 30
Halaman 4 selesai, total artikel terkumpul: 40
Halaman 5 selesai, total artikel terkumpul: 50
Halaman 6 selesai, total artikel terkumpul: 60
Halaman 7 selesai, total artikel terkumpul: 70
Halaman 8 selesai, total artikel terkumpul: 80
Halaman 9 selesai, total artikel terkumpul: 90
Halaman 10 selesai, total artikel terkumpul: 100
Halaman 11 selesai, total artikel terkumpul: 110
Halaman 12 selesai, total artikel terkumpul: 120
Halaman 13 selesai, total artikel terkumpul: 130
Halaman 14 selesai, total artikel terkumpul: 140
Halaman 15 selesai, total artikel terkumpul: 150
Halaman 16 selesai, total artikel terkumpul: 160
Halaman 17 selesai, total artikel terkumpul: 170
Halaman 18 selesai, total artikel terkumpul: 180
Halaman 19 selesai, total artikel terkumpul: 190
Halaman 20 selesai, total artikel terkumpul: 200
Halaman 21 selesai, total artikel terk

In [3]:
import pandas as pd

# Simpan data ke dalam DataFrame
df = pd.DataFrame({"title": titles_list})

# Simpan ke file CSV
df.to_csv("judul_scholar.csv", index=False, encoding="utf-8")

print("Data berhasil disimpan ke judul_scholar.csv")


Data berhasil disimpan ke judul_scholar.csv


In [ ]:
df.head()